In [2]:
import pandas as pd
from fbprophet import Prophet 

In [29]:
train = pd.read_csv('/content/강릉_knn_train.csv')
train['yyyymmddhhnn'] = pd.to_datetime(train['yyyymmddhhnn'])
df = pd.DataFrame()
df['ds'] = train['yyyymmddhhnn']
df['y'] = train['uv']
df['cap'] = 11.3
df['floor'] = 0.0

In [28]:
df

,ds,y,cap,floor
0,2020-01-01 00:00:00,0.0,11.3,0.0
1,2020-01-01 00:10:00,0.0,11.3,0.0
2,2020-01-01 00:20:00,0.0,11.3,0.0
3,2020-01-01 00:30:00,0.0,11.3,0.0
4,2020-01-01 00:40:00,0.0,11.3,0.0
...,...,...,...,...
105259,2021-12-31 23:10:00,0.0,11.3,0.0
105260,2021-12-31 23:20:00,0.0,11.3,0.0
105261,2021-12-31 23:30:00,0.0,11.3,0.0
105262,2021-12-31 23:40:00,0.0,11.3,0.0


In [23]:
m = Prophet()
m.fit(df)

In [24]:
future = m.make_future_dataframe(periods=144) 
forecast = m.predict(future)

In [31]:
holidays = pd.DataFrame()
holiday=[]
for i in range(len(train)):
  if train['yyyymmddhhnn'].iloc[i].hour == 12:
    holiday.append(str(train.yyyymmddhhnn.iloc[i]))

picktime = pd.DataFrame({
'holiday': 'picktime',
'ds': pd.to_datetime(
    holiday
 ),
})

m = Prophet(
    # trend
    changepoint_prior_scale=0.3,
    growth='logistic',
    # seasonality
    weekly_seasonality=10,
    yearly_seasonality=100,
    daily_seasonality=False,
    seasonality_mode='multiplicative',
    # holiday
    holidays=picktime,
    holidays_prior_scale=15
)

# Specifying Custom Seasonalities
m.add_seasonality(name='monthly', period=30.5, fourier_order=5)

m.fit(df)
forecast = m.predict(future)

ValueError: ignored

In [ ]:
from fbprophet.plot import plot_plotly
import plotly.offline as py
fig = plot_plotly(m, forecast) 
py.iplot(fig)

In [27]:
forecast.describe()

,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,holidays,holidays_lower,holidays_upper,monthly,monthly_lower,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
count,105408.000000,105408.000000,105408.000000,105408.000000,105408.000000,105408.000000,105408.000000,105408.000000,105408.000000,105408.000000,...,105408.000000,105408.000000,105408.000000,105408.000000,105408.000000,105408.000000,105408.0,105408.0,105408.0,105408.000000
mean,0.455286,-0.550769,2.580454,0.453571,0.457175,1.205994,1.205994,1.205994,-0.000023,-0.000023,...,-0.005211,-0.005211,-0.005211,0.000398,0.000398,0.000398,0.0,0.0,0.0,1.015239
std,0.236617,1.510611,1.510304,0.249809,0.238588,0.044606,0.044606,0.044606,0.739767,0.739767,...,2.683251,2.683251,2.683251,0.840186,0.840186,0.840186,0.0,0.0,0.0,1.502128
min,-0.551558,-13.697356,-1.194216,-3.819083,0.020890,0.000000,0.000000,0.000000,-1.910091,-1.910091,...,-3.898416,-3.898416,-3.898416,-1.819801,-1.819801,-1.819801,0.0,0.0,0.0,-2.650083
25%,0.241930,-1.685401,1.441072,0.241930,0.242796,1.207644,1.207644,1.207644,-0.361728,-0.361728,...,-2.695975,-2.695975,-2.695975,-0.615781,-0.615781,-0.615781,0.0,0.0,0.0,-0.123184
50%,0.446142,-0.887062,2.239730,0.446142,0.446797,1.207644,1.207644,1.207644,0.196626,0.196626,...,0.002994,0.002994,0.002994,-0.088291,-0.088291,-0.088291,0.0,0.0,0.0,0.678100
75%,0.648470,0.363518,3.492968,0.648470,0.648640,1.207644,1.207644,1.207644,0.418207,0.418207,...,2.667866,2.667866,2.667866,0.457386,0.457386,0.457386,0.0,0.0,0.0,1.925424
max,1.037601,5.305601,18.055410,1.037601,2.933504,1.207644,1.207644,1.207644,1.354972,1.354972,...,4.140621,4.140621,4.140621,2.803995,2.803995,2.803995,0.0,0.0,0.0,6.716252


In [21]:
train['2020-06-01 00:00:00':'2020-06-25 23:50:00'].uv.describe() #1.927639,11.3

count    3600.000000
mean        1.927639
std         2.879717
min         0.000000
25%         0.000000
50%         0.200000
75%         3.000000
max        11.300000
Name: uv, dtype: float64